This notebook is a short and sweet demo showing how we can use roboscheduler's cadence methods to load a set of cadences from a .cfg file, and write them out as a FITS file. 

In the future, we may want to edit to show how we can even load them straight into targetdb, but not yet.

First step - import packages we'll need:

In [54]:
##### import roboscheduler, and its cadence method
import roboscheduler
from roboscheduler import cadence

##### also import fitsio, to write out the cadences to a FITS file.
import fitsio
import csv

##### and import sdssdb to load back to the database
import sdssdb
from sdssdb.peewee.sdss5db import database

##### import numpy to construct the arrays for the rm cadence
import numpy as np

In [55]:
!which roboscheduler

/Users/coveyk/opt/anaconda3/envs/roboscheduler/bin/roboscheduler


next step: open a blank cadence list that we'll use to hold the cadences

In [56]:
clist = cadence.CadenceList()
clist.reset()

Now, use the 'fromcfg' method to import the cadences from the existing [defaultCadences.cfg](https://github.com/sdss/roboDocs/blob/main/cadences/defaultCadences.cfg) file

In [57]:
clist.fromcfg('defaultCadences_v2.cfg')
### uncomment if you want to convince yourself that bright_8x1 and bright_8x4 are incompatible as 
### recorded in brokenCadences_v0.5.cfg
#clist.fromcfg('brokenCadences_v0.5.cfg')

now export the cadences to an array, and print out the first element in that array to check that it makes sense.

In [58]:
cadenceArray = clist.toarray()
print(cadenceArray[0])

('bright_1x1_v2', 1, [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], [b'bright_time', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b'', b''], [15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], [-3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], [8., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], 'bright_1x1', '_v2')


Despite the large number of zeros, that is actually what we'd expect -- that cadence format consists of:

the cadence name: 'bright_1x1'

the number of epochs: 1

the array of delta values associated with each epoch: '[ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

(the nan values are placeholders to signal that the epoch is not active/meaningful, and thus to ignore all the delta min/max etc. info)

the array of sky brightness limits for each epoch: '[1., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

the array of delta_min values for each epoch: '[0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

the array of delta_max values for each epoch: '[0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

the array listing the number of exposures requested for each epoch: '[1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

the array of max_length values for each epoch: '[0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]'

Why are the arrays so long for a 1x1 cadence?  Because the toarray() method has to size the array according to the longest cadence it contains (in this case, a bright_18x4 cadence, so all arrays have spaces for 18 epochs, even if most of those epochs are non-active)

Now let's check some cadences for consistency.

In [59]:
print(cadenceArray[17])
print(cadenceArray[19])
clist.cadence_consistency('bright_8x1_v2', 'bright_8x4_v2')

('bright_8x1_v2', 8, [  0.,  30.,  30.,   3.,  27., 270.,  30.,  30.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.], [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [   0., 1800., 1800., 1800., 1800., 1800., 1800., 1800.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.], [0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], [b'bright_time', b'bright_time', b'bright_time', b'bright_time', b'bright_time', b'bright_time', b'bright_time', b'bright_time', b'', b'', b'', b'', b'', b'', b'', b'', b'', b''], [15., 15., 15., 15., 15., 15., 15., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], [-3., -3., -3., -3., -3., -3., -3., -3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], [8., 8., 8., 8., 8., 8., 8., 8., 0., 0., 0., 0., 0., 0., 0., 0.

(True, [[0, 1, 2, 3, 4, 5, 6, 7]])

In [60]:
clist.cadence_consistency('bright_9x1_v2', 'dark_10x4_4yr_v2')

(True,
 [[0, 1, 2, 3, 4, 5, 6, 7, 8],
  [0, 1, 2, 3, 4, 5, 6, 7, 9],
  [0, 1, 2, 3, 4, 5, 6, 8, 9],
  [0, 1, 2, 3, 4, 5, 7, 8, 9],
  [0, 1, 2, 3, 4, 6, 7, 8, 9],
  [0, 1, 2, 3, 5, 6, 7, 8, 9],
  [0, 1, 2, 4, 5, 6, 7, 8, 9],
  [0, 1, 3, 4, 5, 6, 7, 8, 9],
  [0, 2, 3, 4, 5, 6, 7, 8, 9],
  [1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [61]:
clist.cadence_consistency('bright_18x1_v2', 'mixed2_single_22x1_v2')

KeyError: 'mixed2_single_22x1_v2'

ah, right -- to keep the file simple, we only have the target level cadences in the defaultCadences_v0.5.cfg file.  

If we want to load the field level cadences like the mixed2_single or bright_single cadences, we'll need to load those separately.  

Thankfully Joleen Carlberg has written a handy script (mixed2_single_cadence_generator.py) that we can use to generate the mixed2_single_cadences_v0.5.1.cfg file.  Let's read that in:

In [62]:
clist.fromcfg('mixed2_single_cadences_v2.cfg')

now let's re-do that test from above...

In [63]:
clist.cadence_consistency('bright_18x1_v2', 'mixed2_single_22x1_v2')

(True,
 [[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
  [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
  [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20],
  [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21],
  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20],
  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21]])

and it works now!  Great.  Let's also read in the bright single cadences (which we can generate using a only slightly hacked version of Joleen's script, saved in bright_single_cadence_generator.py):

In [64]:
clist.fromcfg('bright_single_cadences_v2.cfg')

and now let's test that the bright single epochs fit within a mixed2 cadence that is slightly longer (since the bright cadence won't fit in the dark epochs of the mixed cadence).

In [65]:
clist.cadence_consistency('bright_single_20x1_v2', 'mixed2_single_22x1_v2')

(True,
 [[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]])

now let's set up a default RM cadence.  We'll try to make this follow a front-loaded cadence structure, just to show how we can make a non-uniform cadence, but robostrategy won't sweat the details, as long as the number of epochs are correct.

In [66]:
def RM_obsmode(n_epochs, obsmode):
    
    obsmodes = np.array(obsmode, dtype=str)
    
    for i in range(n_epochs-1):
        obsmodes = np.append(obsmodes, obsmode)
        #print(obsmodes)
    #print(len(obsmodes))
    return obsmodes

In [67]:
#begin building a default (front-loaded) RM cadence

rm_label_root = 'dark_174x8'
rm_label_version = '_v2'

rm_obsmodes = RM_obsmode(174, 'dark_rm')

#set up the skybrightness and nexp constraints (which are always 0.35 and 8, respectively)
RM_bright = np.ones(174)*0.35
RM_exps = np.ones(174, dtype=np.int32)*8
RM_min_moon_sep = np.ones(174, dtype=np.int32)*35
RM_deltav_ks91 = np.ones(174)*(-1.5)
RM_twilight_ang = np.ones(174, dtype=np.int32)*15
RM_max_airmass = np.ones(174)*1.4

print(RM_exps)
print(RM_min_moon_sep)
print(RM_deltav_ks91)

[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35]
[-1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5
 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.5 -1.

In [68]:
def RM_timing(n_dense, dense_gap, n_sparse, sparse_gap, seasonal_gap):
    
    #initially, n_dense = 61, dense_gap = 3., n_sparse = 22, sparse_gap = 7, seasonal gap = 183
    
    #set up a season of epochs separated by three days 
    #(in the future we'll have to add gaps for bright runs, but let's keep it simple for now)
    RMdeltas_denselyPacked = np.ones(n_dense)*dense_gap
    RMdeltaMins_denselyPacked = np.ones(n_dense)*dense_gap/2.
    RMdeltaMaxes_denselyPacked = np.ones(n_dense)*dense_gap*1.5
    
    #add on the seasonal gap
    RMdeltas = np.append(RMdeltas_denselyPacked, seasonal_gap)
    RMdeltaMins = np.append(RMdeltaMins_denselyPacked, seasonal_gap-3)
    RMdeltaMaxes = np.append(RMdeltaMaxes_denselyPacked, seasonal_gap+3)

    #now add on another season (with the gap!))
    RMdeltas = np.append(RMdeltas, RMdeltas)
    RMdeltaMins = np.append(RMdeltaMins, RMdeltaMins)
    RMdeltaMaxes = np.append(RMdeltaMaxes, RMdeltaMaxes)

    #now fill in the maxlengths for both seasons (123 epochs)
    maxlengths = np.ones((n_dense+1)*2)*(dense_gap-0.5)
    
    
    #now add on some sparser seasons
    RMdeltas_looselyPacked = np.ones(n_sparse)*sparse_gap
    RMdeltaMins_looselyPacked = np.ones(n_sparse)*sparse_gap/2.
    RMdeltaMaxes_looselyPacked = np.ones(n_sparse)*(sparse_gap+3)

    #and append first of two sparsely packed seasons
    RMdeltas = np.append(RMdeltas, RMdeltas_looselyPacked)
    RMdeltaMins = np.append(RMdeltaMins, RMdeltaMins_looselyPacked)
    RMdeltaMaxes = np.append(RMdeltaMaxes, RMdeltaMaxes_looselyPacked)

    #add on the seasonal gap
    RMdeltas = np.append(RMdeltas, seasonal_gap)
    RMdeltaMins = np.append(RMdeltaMins, seasonal_gap-3)
    RMdeltaMaxes = np.append(RMdeltaMaxes, seasonal_gap+3)

    #and append another sparsely packed season
    RMdeltas = np.append(RMdeltas, RMdeltas_looselyPacked)
    RMdeltaMins = np.append(RMdeltaMins, RMdeltaMins_looselyPacked)
    RMdeltaMaxes = np.append(RMdeltaMaxes, RMdeltaMaxes_looselyPacked)

    #add on the seasonal gap
    RMdeltas = np.append(RMdeltas, seasonal_gap)
    RMdeltaMins = np.append(RMdeltaMins, seasonal_gap-3)
    RMdeltaMaxes = np.append(RMdeltaMaxes, seasonal_gap+3)

    #add the longer max lengths
    more_maxlengths = np.ones( (n_sparse+1)*2)*(dense_gap-0.5)
    RM_maxlengths = np.append(maxlengths, more_maxlengths)

    
    #and fill out the rest of the array
    n_tail = 4
    tail_gap = 30.
    
    #and finally another 4 epochs spaced one month apart
    RMdeltas_loosestPacked = np.ones(n_tail)*tail_gap
    RMdeltaMins_loosestPacked = np.ones(n_tail)*tail_gap/2.
    RMdeltaMaxes_loosestPacked = np.ones(n_tail)*tail_gap*1.5

    #and append the last season
    RMdeltas = np.append(RMdeltas, RMdeltas_loosestPacked)
    RMdeltaMins = np.append(RMdeltaMins, RMdeltaMins_loosestPacked)
    RMdeltaMaxes = np.append(RMdeltaMaxes, RMdeltaMaxes_loosestPacked)

    last_maxlengths = np.ones(n_tail)*(dense_gap-0.5)
    RM_maxlengths = np.append(RM_maxlengths, last_maxlengths)

    #check that everything turned out the right length
    #print(len(RM_maxlengths))
    #print(len(RMdeltas))
    
    return RMdeltas, RMdeltaMins, RMdeltaMaxes, RM_maxlengths

In [69]:
n_dense = 61
dense_gap = 3.0
n_sparse = 22
sparse_gap = 7.
seasonal_gap = 183.

RMdeltas, RMdeltaMins, RMdeltaMaxes, RM_maxlengths = RM_timing(n_dense, dense_gap, n_sparse, sparse_gap, seasonal_gap)

In [70]:
#now let's add this RM cadence to our cadence list
clist.add_cadence(name=rm_label_root+rm_label_version, nepochs = 174, nexp = RM_exps, skybrightness = RM_bright, 
                            delta = RMdeltas, delta_min = RMdeltaMins, delta_max = RMdeltaMaxes, 
                            max_length = RM_maxlengths, obsmode_pk = rm_obsmodes, 
                            label_root = rm_label_root, label_version = rm_label_version, 
                            min_moon_sep = RM_min_moon_sep, min_deltav_ks91 = RM_deltav_ks91, 
                            min_twilight_ang = RM_twilight_ang, max_airmass = RM_max_airmass)

In [71]:
clist.cadence_consistency('dark_10x4_4yr_v2', 'dark_174x8_v2')

(False, [])

In [72]:
#now lets add an RM lite cadence

rm_lite_label_root = 'dark_100x8'
rm_lite_label_version = '_v2'

rm_lite_obsmodes = RM_obsmode(100, 'dark_rm')

#set up the skybrightness and nexp constraints (which are always 0.35 and 8, respectively)
RM_lite_bright = np.ones(100)*0.35
RM_lite_exps = np.ones(100, dtype=np.int32)*8

RM_lite_min_moon_sep = np.ones(100, dtype=np.int32)*35
RM_lite_deltav_ks91 = np.ones(100)*(-1.5)
RM_lite_twilight_ang = np.ones(100, dtype=np.int32)*15
RM_lite_max_airmass = np.ones(100)*1.4

#set up the timing constraints
n_lite_dense = 32
dense_lite_gap = 3.0
n_lite_sparse = 14
sparse_lite_gap = 10.
seasonal_lite_gap = 183.

#fill out the arrays
RMdeltas_lite, RMdeltaMins_lite, RMdeltaMaxes_lite, RM_maxlengths_lite = RM_timing(n_lite_dense, dense_lite_gap, n_lite_sparse, sparse_lite_gap, seasonal_lite_gap)

#now let's add this RM cadence to our cadence list
clist.add_cadence(name=rm_lite_label_root+rm_lite_label_version, nepochs = 100, nexp = RM_lite_exps, 
                  skybrightness = RM_lite_bright, delta = RMdeltas_lite, delta_min = RMdeltaMins_lite, 
                  delta_max = RMdeltaMaxes_lite, max_length = RM_maxlengths_lite, obsmode_pk = rm_lite_obsmodes, 
                            label_root = rm_lite_label_root, label_version = rm_lite_label_version, 
                            min_moon_sep = RM_lite_min_moon_sep, min_deltav_ks91 = RM_lite_deltav_ks91, 
                            min_twilight_ang = RM_lite_twilight_ang, max_airmass = RM_lite_max_airmass)

In [73]:
clist.cadence_consistency('dark_100x8_v2', 'dark_174x8_v2')

(False, [])

In [74]:
clist.cadence_consistency('bright_5x1_v2', 'dark_10x4_4yr_v2')

(True,
 [[0, 1, 2, 3, 4],
  [0, 1, 2, 3, 5],
  [0, 1, 2, 3, 6],
  [0, 1, 2, 3, 7],
  [0, 1, 2, 3, 8],
  [0, 1, 2, 3, 9],
  [0, 1, 2, 4, 5],
  [0, 1, 2, 4, 6],
  [0, 1, 2, 4, 7],
  [0, 1, 2, 4, 8],
  [0, 1, 2, 4, 9],
  [0, 1, 2, 5, 6],
  [0, 1, 2, 5, 7],
  [0, 1, 2, 5, 8],
  [0, 1, 2, 5, 9],
  [0, 1, 2, 6, 7],
  [0, 1, 2, 6, 8],
  [0, 1, 2, 6, 9],
  [0, 1, 2, 7, 8],
  [0, 1, 2, 7, 9],
  [0, 1, 2, 8, 9],
  [0, 1, 3, 4, 5],
  [0, 1, 3, 4, 6],
  [0, 1, 3, 4, 7],
  [0, 1, 3, 4, 8],
  [0, 1, 3, 4, 9],
  [0, 1, 3, 5, 6],
  [0, 1, 3, 5, 7],
  [0, 1, 3, 5, 8],
  [0, 1, 3, 5, 9],
  [0, 1, 3, 6, 7],
  [0, 1, 3, 6, 8],
  [0, 1, 3, 6, 9],
  [0, 1, 3, 7, 8],
  [0, 1, 3, 7, 9],
  [0, 1, 3, 8, 9],
  [0, 1, 4, 5, 6],
  [0, 1, 4, 5, 7],
  [0, 1, 4, 5, 8],
  [0, 1, 4, 5, 9],
  [0, 1, 4, 6, 7],
  [0, 1, 4, 6, 8],
  [0, 1, 4, 6, 9],
  [0, 1, 4, 7, 8],
  [0, 1, 4, 7, 9],
  [0, 1, 4, 8, 9],
  [0, 1, 5, 6, 7],
  [0, 1, 5, 6, 8],
  [0, 1, 5, 6, 9],
  [0, 1, 5, 7, 8],
  [0, 1, 5, 7, 9],
  [0, 1, 5, 8, 9],
  [0,

Now let's write this cadence array out to a FITS file for easy transportation:

In [75]:
FinalCadenceArray = clist.toarray()

print(len(FinalCadenceArray))

#now let's replace all the non-active zero deltas with -1
print(FinalCadenceArray[39])

#np.shape(FinalCadenceArray)
#TrimmedArray = np.delete(FinalCadenceArray, 9, 1)
#print(FinalCadenceArray[41])

238
('dark_3x4_v2', 3, [  0., 365., 365.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.

In [76]:
fitsio.write("DefaultCadences_v2.fits", FinalCadenceArray)

Success!  

This command should produce the [DefaultCadences.fits file](https://github.com/sdss/roboDocs/blob/main/cadences/DefaultCadences.fits) that lives next to this notebook.

As telegraphed above, we could then use some of the functionality roboscheduler provides to load these cadences into the targeting database, but that would require getting credentials etc. correct, so let's leave that for later...

Now let's print to a csv so that Pramod can import easily...

In [77]:
clist.tocsv('defaultCadences_v2.csv')